In [5]:
'''
功能: 在資料庫已經有資料下，新增個股的資料
'''



import csv
import pprint
import datetime
import copy
import re
from pymongo.mongo_client import MongoClient

pp = pprint.PrettyPrinter(indent = 4)

filename = './data/1207-1208.csv'
dateFormat = '%Y/%m/%d'

### db config ###
client = MongoClient()
db = client['Stock']
stockCollec = db['price']

In [9]:
def isStockInDB(code):
    return stockCollec.find({'code': code}).count() > 0

def pack_price(line):
    return {
        'date': datetime.datetime.strptime(line['年月日'], dateFormat),
        'open': float(re.sub(',', '', line['開盤價(元)'])),
        'high': float(re.sub(',', '', line['最高價(元)'])),
        'low': float(re.sub(',', '', line['最低價(元)'])),
        'close': float(re.sub(',', '', line['收盤價(元)'])),
        'volumn': float(re.sub(',', '', line['成交量(千股)']))
    }

with open(filename, 'r') as f:
    reader = csv.DictReader(f)
    
    for i, line in enumerate(reader):
        code = line['證券代碼'].split(' ')[0]
        
        if isStockInDB(code):
            thisPrice = pack_price(line)
            stockCollec.update({'code': code}, {'$push': {
                'history': {
                    '$each': [copy.deepcopy(thisPrice)],
                    '$sort': {'date': 1}
            }}})
        else:
            thisStock = {'code': code, 'history': [pack_price(line)]}
            stockCollec.insert_one(copy.deepcopy(thisStock))

c:\program files (x86)\python36-32\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
